In [10]:
# D&D chromosomes (from Munroes What if...)
import random

def ddstat(): # strict ;)
    '''roll stat dice'''
    return random.randint(1,6) + random.randint(1,6) + random.randint(1,6)

def createcset(l):
    '''create a a character set: either a stat value or a multiplier'''
    res = []
    for i in range(l-1):
        if random.random() > 0.8:
            res.append(-random.uniform(0.5, 2.5)) # multiplier
        else:
            res.append(ddstat())
    res.append(random.choice([-1, 1])) # last stat is sex (-1 female, 1 male)
    return res

def combine(cset0, cset1):
    '''combine two character sets'''
    if len(cset0) != len(cset1):
        return -1
    res = []
    for ic in range(len(cset0)-1):
        if cset0[ic] < 0: # multipliers are negative
            if cset1[ic] < 0:
                res.append(1) # multiplier on both sides: you get a 1
            else:
                res.append(int(-cset0[ic] * cset1[ic]))
        else:
            if cset1[ic] < 0:
                res.append(int(-cset0[ic] * cset1[ic]))
            else:
                res.append(max(cset0[ic], cset1[ic]))
    if cset0[len(cset0)-1] < 0:
        if cset1[len(cset1)-1] < 0:
            res.append(-1) # female
        else:
            if random.random() > 0.5:
                res.append(1)
            else:
                res.append(-1)
    else:
        if cset1[len(cset1)-1] < 0:
            if random.random() > 0.5:
                res.append(1)
            else:
                res.append(-1)
        else:
            res.append(1) # male
    return res

if __name__=="__main__":
    print(combine([15, -2, -1, 12, -0.5, 14, -1], [5, 12, 14, -1.5, 14, 15, -1]))
    print(ddstat())
    print(createcset(7))
    print(combine(createcset(7), createcset(7)))

[15, 24, 14, 18, 7, 15, -1]
15
[-1.4079717394445106, 15, 7, 11, 7, -1.4191107472943223, 1]
[9, 9, 8, 13, 11, 21, 1]


In [67]:
# https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
'''
#Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
**Summary of the algorithm**
- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
**Data download**
[English to French sentence pairs.
](http://www.manythings.org/anki/fra-eng.zip)
[Lots of neat sentence pairs datasets.
](http://www.manythings.org/anki/)
**References**
- [Sequence to Sequence Learning with Neural Networks
   ](https://arxiv.org/abs/1409.3215)
- [Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    ](https://arxiv.org/abs/1406.1078)
'''
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

import os

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = r'fra.txt'
os.chdir(r'D:\BernieData\DeepL')

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 16
Max sequence length for outputs: 59
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 20s 3ms/step - loss: 1.1668 - accuracy: 0.7259 - val_loss: 1.0686 - val_accuracy: 0.7150
Epoch 2/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.8533 - accuracy: 0.7707 - val_loss: 0.8382 - val_accuracy: 0.7666
Epoch 3/100
8000/8000 [==============================] - 16s 2ms/step - loss: 0.6931 - accuracy: 0.8041 - val_loss: 0.7161 - val_accuracy: 0.7923
Epoch 4/100
8000/8000 [==============================] - 16s 2ms/step - loss: 0.6032 - accuracy: 0.8251 - val_loss: 0.6617 - val_accuracy: 0.8044
Epoch 5/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.5533 - accuracy: 0.8380 - val_loss: 0.6154 - val_accuracy: 0.8193
Epoch 6/100
8000/8000 [==============================] - 

-
Input sentence: Go now.
Decoded sentence: Vas-y maintenant.

-
Input sentence: Go now.
Decoded sentence: Vas-y maintenant.

-
Input sentence: Go now.
Decoded sentence: Vas-y maintenant.

-
Input sentence: Got it!
Decoded sentence: Compris !

-
Input sentence: Got it!
Decoded sentence: Compris !

-
Input sentence: Got it?
Decoded sentence: Pigé ?

-
Input sentence: Got it?
Decoded sentence: Pigé ?

-
Input sentence: Got it?
Decoded sentence: Pigé ?

-
Input sentence: Hop in.
Decoded sentence: Montez.

-
Input sentence: Hop in.
Decoded sentence: Montez.

-
Input sentence: Hug me.
Decoded sentence: Serre-moi dans tes bras !

-
Input sentence: Hug me.
Decoded sentence: Serre-moi dans tes bras !

-
Input sentence: I fell.
Decoded sentence: Je suis tombé.

-
Input sentence: I fell.
Decoded sentence: Je suis tombé.

-
Input sentence: I know.
Decoded sentence: Je sais.

-
Input sentence: I left.
Decoded sentence: Je suis parti.

-
Input sentence: I left.
Decoded sentence: Je suis parti.

-
I

In [15]:
# https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
'''
#Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
**Summary of the algorithm**
- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
**Data download**
[English to French sentence pairs.
](http://www.manythings.org/anki/fra-eng.zip)
[Lots of neat sentence pairs datasets.
](http://www.manythings.org/anki/)
**References**
- [Sequence to Sequence Learning with Neural Networks
   ](https://arxiv.org/abs/1409.3215)
- [Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    ](https://arxiv.org/abs/1406.1078)
'''
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, GRU, Dense
import numpy as np

import os

batch_size = 64  # Batch size for training.
epochs = 1  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = r'fra.txt'
os.chdir(r'D:\BernieData\DeepL')

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

'''Turn the sentences into 3 Numpy arrays, encoder_input_data, decoder_input_data, decoder_target_data:
encoder_input_data is a 3D array of shape (num_pairs, max_english_sentence_length, num_english_characters)
    containing a one-hot vectorization of the English sentences.
decoder_input_data is a 3D array of shape (num_pairs, max_french_sentence_length, num_french_characters)
    containg a one-hot vectorization of the French sentences.
decoder_target_data is the same as decoder_input_data but offset by one timestep.
decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :].'''
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)): # over input corpus
    for t, char in enumerate(input_text): # over characters in input text
        encoder_input_data[i, t, input_token_index[char]] = 1. # char to char token
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = GRU(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h = encoder(encoder_inputs)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_gru = GRU(latent_dim, return_sequences=True)
decoder_outputs = decoder_gru(decoder_inputs, initial_state=state_h)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, state_h)

decoder_state_input_h = Input(shape=(latent_dim,))
#decoder_states_inputs = [decoder_state_input_h]
decoder_states_inputs = Input(shape=(latent_dim,))
#decoder_outputs, state_h = decoder_gru(decoder_inputs, initial_state=decoder_states_inputs)
print(np.shape(decoder_inputs))
decoder_outputs, state_h = decoder_gru(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to something readable
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences (to simplify, here we assume a batch of size 1)
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop character
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 16
Max sequence length for outputs: 59
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 17s 2ms/step - loss: 1.2189 - accuracy: 0.7267 - val_loss: 0.9825 - val_accuracy: 0.7214
(None, None, 92)


TypeError: Cannot iterate over a tensor with unknown first dimension.

In [4]:
# from Akira Takezawa and adapted quite heavily
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import os
import re

MAX_LEN = 20 # to unify the length of the input sentences
VOCAB_SIZE = 20 # to decide the dimension of sentence’s one-hot vector
EMBEDDING_DIM = 20 # to decide the dimension of Word2Vec

def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    return text

def tagger(decoder_input_sentence):
    bos = "<BOS> " # begin of sequence
    eos = " <EOS>" # end of sequence
    final_target = [bos + text + eos for text in decoder_input_sentence]
    return final_target

def vocab_creator(text_lists, VOCAB_SIZE):
    '''create vocabulary and return wrd-id and id-wrd mappings'''
    tokenizer = Tokenizer(num_words=VOCAB_SIZE)
    tokenizer.fit_on_texts(text_lists)
    dictionary = tokenizer.word_index
    word2idx = {}
    idx2word = {}
    for k, v in dictionary.items():
        if v < VOCAB_SIZE:
            word2idx[k] = v
            idx2word[v] = k
        if v >= VOCAB_SIZE-1:
            continue
    return word2idx, idx2word

def txt2seq(txt, vocab):
    '''tokenize text and return list of tokens'''
    res = []
    for sent in txt: # txt is a list of sentences
        res.append([vocab[0][wrd] for wrd in sent.split()])
    return res

def text2seq(encoder_text, decoder_text, vocab):
    #tokenizer = Tokenizer(num_words=VOCAB_SIZE)
    #encoder_sequences = tokenizer.texts_to_sequences(encoder_text)
    #decoder_sequences = tokenizer.texts_to_sequences(decoder_text)
    encoder_sequences = txt2seq(encoder_text, vocab)
    decoder_sequences = txt2seq(decoder_text, vocab)
    return encoder_sequences, decoder_sequences

def padding(encoder_sequences, decoder_sequences, MAX_LEN):
    encoder_input_data = pad_sequences(encoder_sequences, maxlen=MAX_LEN, dtype='int32', padding='post', truncating='post')
    decoder_input_data = pad_sequences(decoder_sequences, maxlen=MAX_LEN, dtype='int32', padding='post', truncating='post')
    return encoder_input_data, decoder_input_data

# GLOVE_DIR = path for glove.6B.100d.txt
def glove_100d_dictionary(GLOVE_DIR):
    embeddings_index = {}
    f = open(os.path.join(GLOVE_DIR, r'glove.6B.100d.txt'), encoding="utf-8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

# this time: embedding_dimension = 100d
def embedding_matrix_creator(word_index, embeddings_index, embedding_dimension):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimension))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

# MAX_LEN = 20
# num_samples = len(encoder_sequences)
# VOCAB_SIZE = 15000

def decoder_output_creator(decoder_input_data, num_samples, MAX_LEN, VOCAB_SIZE):
    '''reshape to (num_samples, MAX_LEN, VOCAB_SIZE)'''
    print('vocab: ', VOCAB_SIZE)
    decoder_output_data = np.zeros((num_samples, MAX_LEN, VOCAB_SIZE), dtype="float32") # empty output array
    for i, seqs in enumerate(decoder_input_data): # iterate over samples - i till num_samples
        for j, seq in enumerate(seqs): # iterate over tokens in current sequence - j till MAX_LEN
            if j > 0:
                decoder_output_data[i][j][seq] = 1.
    print(decoder_output_data.shape)
    return decoder_output_data

def data_splitter(encoder_input_data, decoder_input_data, test_size1=0.2, test_size2=0.3):
    en_train, en_test, de_train, de_test = train_test_split(encoder_input_data, decoder_input_data, test_size=test_size1)
    en_train, en_val, de_train, de_val = train_test_split(en_train, de_train, test_size=test_size2)
    return en_train, en_val, en_test, de_train, de_val, de_test

def embedding_layer_creator(VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN, embedding_matrix):
    embedding_layer = Embedding(input_dim = VOCAB_SIZE, 
                              output_dim = EMBEDDING_DIM,
                              input_length = MAX_LEN,
                              weights = [embedding_matrix],
                              trainable = False)
    return embedding_layer

def seq2seq_model_builder(encoder_data, decoder_data, MAX_LEN, VOCAB_SIZE, HIDDEN_DIM=300):
    
    '''
encoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
encoder_embedding = embed_layer(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
decoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
decoder_embedding = embed_layer(decoder_inputs)
decoder_LSTM = LSTM(HIDDEN_DIM, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])
# dense_layer = Dense(VOCAB_SIZE, activation='softmax')
outputs = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], outputs)'''
    encoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
    # model.add(tf.keras.layers.Embedding(1000, 64, input_length=10))
    #word_embedding=Embedding(input_dim=vocab_size,output_dim=8,input_length=maxlen)(word_input)
    # https://www.kaggle.com/rajmehra03/a-detailed-explanation-of-keras-embedding-layer
    encoder_embedding = Embedding(input_dim=VOCAB_SIZE, output_dim=64, input_length=MAX_LEN)(encoder_inputs)
    # encoder_embedding = embed_layer(encoder_inputs)
    encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
    encoder_outputs, state_h, state_c = encoder_LSTM(encoder_data)
    decoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
    #decoder_embedding = embed_layer(decoder_inputs)
    decoder_embedding = Embedding(input_dim=VOCAB_SIZE, output_dim=64, input_length=MAX_LEN)(decoder_inputs)
    decoder_LSTM = LSTM(HIDDEN_DIM, return_state=True, return_sequences=True)
    decoder_outputs, _, _ = decoder_LSTM(decoder_data, initial_state=[state_h, state_c])
    # dense_layer = Dense(VOCAB_SIZE, activation='softmax')
    outputs = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], outputs)
    return model

'''print(clean_text('this is a test sentence'))
print(tagger(['with the teeth of your zipper', 'so they tell me', 'so which dakota you from']))
print(vocab_creator(['with the teeth of your zipper', 'so they tell me', 'so which dakota you from'], 30))
enc_txt = ['with the teeth of your zipper', 'so they tell me', 'so which dakota you from']
dec_txt = ['with the teeth of your zipper', 'so they tell me', 'so which dakota you from']
my_vocab = vocab_creator(['with the teeth of your zipper', 'so they tell me', 'so which dakota you from'], 30)
enc_seq, dec_seq = text2seq(enc_txt, dec_txt, my_vocab)
print(enc_seq)
enc_seq, dec_seq = padding(enc_seq, dec_seq, 15)
print('enc/dec seq: ', enc_seq, dec_seq)
#embd_idx = glove_100d_dictionary(r'D:\BernieData\DeepL\data')
#embd_mtrx = embedding_matrix_creator(my_vocab[0], embd_idx, 100)
#print(embedding_layer_creator(len(my_vocab[0]), 100, 15, embd_mtrx))
print('dec_seq:', dec_seq, np.shape(dec_seq))
print(decoder_output_creator(dec_seq, 3, 15, len(my_vocab[0])+1))
print(data_splitter(enc_seq, dec_seq))'''

encoder_text = ['with the teeth of your zipper', 'so they tell me', 'so which dakota you from']
decoder_text = ['with the teeth of your zipper', 'so they tell me', 'so which dakota you from']
my_vocab = vocab_creator(text_lists=encoder_text+decoder_text, VOCAB_SIZE=14999)
VOCAB_SIZE = len(my_vocab[0])
num_samples = 15
MAX_LEN = 15
encoder_sequences, decoder_sequences = text2seq(encoder_text, decoder_text, my_vocab)
encoder_input_data, decoder_input_data = padding(encoder_sequences, decoder_sequences, MAX_LEN) 
#decoder_inputs = tagger(decoder_input_data)
decoder_output_data = decoder_output_creator(decoder_input_data, num_samples, MAX_LEN, VOCAB_SIZE+1)
en_train, en_val, en_test, de_train, de_val, de_test = data_splitter(encoder_input_data, decoder_input_data)

#embd_idx = glove_100d_dictionary(r'D:\BernieData\DeepL\data')
#embd_mtrx = embedding_matrix_creator(my_vocab[0], embd_idx, 100)
#embd_lyr = embedding_layer_creator(len(my_vocab[0]), 100, 15, embd_mtrx)

model = seq2seq_model_builder(encoder_input_data, decoder_input_data, MAX_LEN, VOCAB_SIZE)
model.summary()

vocab:  15
(15, 15, 15)


ValueError: Layer lstm_2 was called with an input that isn't a symbolic tensor. Received type: <class 'numpy.ndarray'>. Full input: [array([[ 2,  3,  4,  5,  6,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  8,  9, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1, 11, 12, 13, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])]. All inputs to the layer should be tensors.

In [16]:
# 21 game
import random
import numpy as np

class game21:
    def __init__(self):
        self.prt = 0
    
    def hummv(self):
        mvv = -1
        while mvv > 3 or mvv < 1:
            mvv = int(input('Enter your number (1-3):'))
        self.prt += mvv
        print('human total: ', self.prt)
        if self.prt == 21:
            print('you win!')
            return 0
        if self.prt > 21:
            print('you lose!')
            return 1
        return 2
    
    def cmpmv(self):
        #print(self.minimaxd(self.prt))
        chsmv = self.minimaxd(self.prt)
        print('succ: ', self.successors(self.prt), ' choose ', chsmv)
        self.prt = self.successors(self.prt)[chsmv]
        print('computer total: ', self.prt)
        if self.prt == 21:
            print('computer wins!')
            return 1
        if self.prt > 21:
            print('computer loses!')
            return 0
        return 2
    
    def playgm(self):
        while self.prt < 21:
            if self.hummv() < 2:
                return
            self.cmpmv()
        return
    
    def util(self, state):
        # todo: improve, add forward-looking?
        #print('util: ', state)
        if state >= 21: # terminal state
            return 1
        if state > 14 and state < 18:
            return -5
        return -1
    
    def successors(self, state): # for 21 the state is just the running total
        if state >= 21:
            return []
        if state >= 20:
            return [state+1] # 1 possible move
        if state >= 19:
            return [state+1, state+2] # 2 possible moves
        return [state+1, state+2, state+3] # 3 possible moves
    
    def minimaxd(self, state):
        opts = [self.maxv(sx) for sx in self.successors(state)]
        print('minimaxd: ', state, self.successors(state), ' options: ', opts)
        return np.argmax(opts) # return index af max value of successor states
    
    def maxv(self, state):
        #print('max: ', state, self.successors(state))
        if state >= 21: # terminal state
            return self.util(state)
        v = -100
        for s in self.successors(state):
            v = max(v, self.minv(s))
        return v
    
    def minv(self, state):
        #print('min: ', state, self.successors(state))
        if state >= 21: # terminal state
            return self.util(state)
        v = 100
        for s in self.successors(state):
            v = min(v, self.maxv(s))
        return v
    
    '''function minimax(node, depth, maximizingPlayer) is
    if depth = 0 or node is a terminal node then
        return the heuristic value of node
    if maximizingPlayer then
        value := −∞
        for each child of node do
            value := max(value, minimax(child, depth − 1, FALSE))
        return value
    else: # minimizing player
        value := +∞
        for each child of node do
            value := min(value, minimax(child, depth − 1, TRUE))
        return value'''
    
def main():
    myg = game21()
    myg.playgm()

if __name__=='__main__':
    main()

Enter your number (1-3):3
human total:  3
minimaxd:  3 [4, 5, 6]  options:  [1, 1, 1]
succ:  [4, 5, 6]  choose  0
computer total:  4
Enter your number (1-3):3
human total:  7
minimaxd:  7 [8, 9, 10]  options:  [1, 1, 1]
succ:  [8, 9, 10]  choose  0
computer total:  8
Enter your number (1-3):3
human total:  11
minimaxd:  11 [12, 13, 14]  options:  [1, 1, 1]
succ:  [12, 13, 14]  choose  0
computer total:  12
Enter your number (1-3):3
human total:  15
minimaxd:  15 [16, 17, 18]  options:  [1, 1, 1]
succ:  [16, 17, 18]  choose  0
computer total:  16
Enter your number (1-3):3
human total:  19
minimaxd:  19 [20, 21]  options:  [1, 1]
succ:  [20, 21]  choose  0
computer total:  20
Enter your number (1-3):3
human total:  23
you lose!


In [2]:
# https://www.kaggle.com/rajmehra03/a-detailed-explanation-of-keras-embedding-layer

# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#nltk
import nltk

#stop-words
from nltk.corpus import stopwords
stop_words=set(nltk.corpus.stopwords.words('english'))

# tokenizing
from nltk import word_tokenize,sent_tokenize

#keras
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input
from keras.models import Model

sample_text_1="bitty bought a bit of butter"
sample_text_2="but the bit of butter was a bit bitter"
sample_text_3="so she bought some better butter to make the bitter butter better"

corp=[sample_text_1,sample_text_2,sample_text_3]
no_docs=len(corp)

vocab_size=50 
encod_corp=[]
for i,doc in enumerate(corp):
    encod_corp.append(one_hot(doc,50))
    print("The encoding for document",i+1," is : ",one_hot(doc,50))
    
# length of maximum document. will be nedded whenever create embeddings for the words
maxlen=-1
for doc in corp:
    tokens=nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen=len(tokens)
print("The maximum number of words in any document is : ",maxlen)

# now to create embeddings all of our docs need to be of same length. hence we can pad the docs with zeros.
pad_corp=pad_sequences(encod_corp,maxlen=maxlen,padding='post',value=0.0)
print("No of padded documents: ",len(pad_corp))

for i,doc in enumerate(pad_corp):
     print("The padded encoding for document",i+1," is : ",doc)
        
# specifying the input shape
input=Input(shape=(no_docs,maxlen),dtype='float64')

'''
shape of input. 
each document has 12 element or words which is the value of our maxlen variable.

'''
word_input=Input(shape=(maxlen,),dtype='float64')  

# creating the embedding
word_embedding=Embedding(input_dim=vocab_size,output_dim=8,input_length=maxlen)(word_input)

word_vec=Flatten()(word_embedding) # flatten
embed_model =Model([word_input],word_vec) # combining all into a Keras model

embed_model.compile(optimizer=keras.optimizers.Adam(lr=1e-3),loss='binary_crossentropy',metrics=['acc']) 
# compiling the model. parameters can be tuned as always.

print(type(word_embedding))
print(word_embedding)

print(embed_model.summary()) # summary of the model

embeddings=embed_model.predict(pad_corp) # finally getting the embeddings.

print("Shape of embeddings : ",embeddings.shape)
print(embeddings)

embeddings=embeddings.reshape(-1,maxlen,8)
print("Shape of embeddings : ",embeddings.shape) 
print(embeddings)

for i,doc in enumerate(embeddings):
    for j,word in enumerate(doc):
        print("The encoding for ",j+1,"th word","in",i+1,"th document is : \n\n",word)
        
        

The encoding for document 1  is :  [38, 32, 39, 39, 14, 4]
The encoding for document 2  is :  [30, 33, 39, 14, 4, 26, 39, 39, 39]
The encoding for document 3  is :  [46, 14, 32, 28, 28, 4, 9, 43, 33, 39, 4, 28]
The maximum number of words in any document is :  12
No of padded documents:  3
The padded encoding for document 1  is :  [38 32 39 39 14  4  0  0  0  0  0  0]
The padded encoding for document 2  is :  [30 33 39 14  4 26 39 39 39  0  0  0]
The padded encoding for document 3  is :  [46 14 32 28 28  4  9 43 33 39  4 28]
<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("embedding_2/embedding_lookup/Identity_1:0", shape=(None, 12, 8), dtype=float32)
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 12)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 12, 8)         

In [12]:
# attempt to straighten out the issues with seq2seq
# Keras API: https://keras.io/guides/functional_api/

# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import os
import re

#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#nltk
import nltk

#stop-words
from nltk.corpus import stopwords
stop_words=set(nltk.corpus.stopwords.words('english'))

# tokenizing
from nltk import word_tokenize,sent_tokenize

#keras
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, Input, TimeDistributed
from keras.models import Model

sample_text_1="bitty bought a bit of butter"
sample_text_2="but the bit of butter was a bit bitter"
sample_text_3="so she bought some better butter to make the bitter butter better"

corp = [sample_text_1, sample_text_2, sample_text_3] # corpus
no_docs = len(corp) # number of documents in corpus

vocab_size = 50 # size of vocabulary

encod_corp = []
for i, doc in enumerate(corp): # docs in corpus
    encod_corp.append(one_hot(doc,50))
    print("The encoding for document", i+1, " is : ", one_hot(doc,50))
    
# length of maximum document. will be nedded whenever create embeddings for the words
maxlen = -1
for doc in corp: # docs in corpus
    tokens = nltk.word_tokenize(doc) # tokenize document
    if(maxlen < len(tokens)):
        maxlen = len(tokens)
print("The maximum number of words in any document is : ", maxlen)

# now to create embeddings all of our docs need to be of same length. hence we can pad the docs with zeros.
pad_corp = pad_sequences(encod_corp, maxlen = maxlen, padding = 'post', value = 0.0)
print("No of padded documents: ", len(pad_corp))

for i, doc in enumerate(pad_corp):
     print("The padded encoding for document", i+1, " is : ", doc)

        
print('vocab_size: ', vocab_size)
print('maxlen: ', maxlen)

# clear old model
keras.backend.clear_session()
        
encoder_inputs = Input(shape=(maxlen, ), dtype='int32',)
print('encoder_inputs: ', np.shape(encoder_inputs)) # (None, 12)
#encoder_embedding = embed_layer(encoder_inputs)
encoder_embedding = Embedding(input_dim=vocab_size, output_dim=8, input_length=maxlen)(encoder_inputs)
print('encoder_embedding: ', np.shape(encoder_embedding)) # (None, 12, 8)
HIDDEN_DIM = 3
encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)

encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
print('encoder_outputs: ', np.shape(encoder_outputs)) # (None, 3)
print('state_h: ', np.shape(state_h)) # (None, 3)
print('state_c: ', np.shape(state_c)) # (None, 3)
decoder_inputs = Input(shape=(maxlen, ), dtype='int32',)
print('decoder_inputs: ', np.shape(decoder_inputs)) # (None, 12)
#decoder_embedding = embed_layer(decoder_inputs)
decoder_embedding = Embedding(input_dim=vocab_size, output_dim=8, input_length=maxlen)(decoder_inputs)
print('decoder_embedding: ', np.shape(decoder_embedding)) # (None, 12, 8)
decoder_LSTM = LSTM(HIDDEN_DIM, return_state=True, return_sequences=True)

decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c]) # no need to return hidden states
print('dec_out: ', np.shape(decoder_outputs)) # (None, 12, 3)

# dense_layer = Dense(VOCAB_SIZE, activation='softmax')
outputs = TimeDistributed(Dense(vocab_size, activation='softmax'))(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], outputs)
model.summary()
#keras.utils.plot_model(model, "seq2seq_t0.png", show_shapes=True)

print(outputs.shape)

print('outputs: ', np.shape(outputs)) # (None, 12, 50)


data_path = 'D:\BernieData\DeepL\data'
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

#checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}.hdf5', verbose=1)

'''model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,
                        validation_data=valid_data_generator.generate(),
                        validation_steps=len(valid_data) // (batch_size*num_steps), callbacks=[checkpointer])'''

# Actually training the model: (x=input data, y=target data)
# see https://www.tensorflow.org/api_docs/python/tf/keras/Model
model.fit([pad_corp, pad_corp], outputs, epochs=1)

'''model = load_model(data_path + "\model-40.hdf5")
dummy_iters = 40
example_training_generator = KerasBatchGenerator(train_data, num_steps, 1, vocabulary,
                                                     skip_step=1)
print("Training data:")
for i in range(dummy_iters):
    dummy = next(example_training_generator.generate())
num_predict = 10
true_print_out = "Actual words: "
pred_print_out = "Predicted words: "
for i in range(num_predict):
    data = next(example_training_generator.generate())
    prediction = model.predict(data[0])
    predict_word = np.argmax(prediction[:, num_steps-1, :])
    true_print_out += reversed_dictionary[train_data[num_steps + dummy_iters + i]] + " "
    pred_print_out += reversed_dictionary[predict_word] + " "
print(true_print_out)
print(pred_print_out)'''

The encoding for document 1  is :  [23, 39, 14, 42, 11, 7]
The encoding for document 2  is :  [8, 20, 42, 11, 7, 45, 14, 42, 4]
The encoding for document 3  is :  [44, 49, 39, 34, 11, 7, 5, 32, 20, 4, 7, 11]
The maximum number of words in any document is :  12
No of padded documents:  3
The padded encoding for document 1  is :  [23 39 14 42 11  7  0  0  0  0  0  0]
The padded encoding for document 2  is :  [ 8 20 42 11  7 45 14 42  4  0  0  0]
The padded encoding for document 3  is :  [44 49 39 34 11  7  5 32 20  4  7 11]
vocab_size:  50
maxlen:  12
encoder_inputs:  (None, 12)
encoder_embedding:  (None, 12, 8)
encoder_outputs:  (None, 3)
state_h:  (None, 3)
state_c:  (None, 3)
decoder_inputs:  (None, 12)
decoder_embedding:  (None, 12, 8)
dec_out:  (None, 12, 3)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (Inpu

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [15]:
# https://keras.io/getting_started/intro_to_keras_for_researchers/

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):
    """Maps MNIST digits to a triplet (z_mean, z_log_var, z)."""

    def __init__(self, latent_dim=32, intermediate_dim=64, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation=tf.nn.relu)
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z
    
class Decoder(layers.Layer):
    """Converts z, the encoded digit vector, back into a readable digit."""

    def __init__(self, original_dim, intermediate_dim=64, **kwargs):
        super(Decoder, self).__init__(**kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation=tf.nn.relu)
        self.dense_output = layers.Dense(original_dim, activation=tf.nn.sigmoid)

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)
    
class VariationalAutoEncoder(layers.Layer):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(self, original_dim, intermediate_dim=64, latent_dim=32, **kwargs):
        super(VariationalAutoEncoder, self).__init__(**kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed
    
# Our model.
vae = VariationalAutoEncoder(original_dim=784, intermediate_dim=64, latent_dim=32)

# Loss and optimizer.
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Prepare a dataset.
(x_train, _), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    x_train.reshape(60000, 784).astype("float32") / 255
)
dataset = dataset.shuffle(buffer_size=1024).batch(32)


@tf.function
def training_step(x):
    with tf.GradientTape() as tape:
        reconstructed = vae(x)  # Compute input reconstruction.
        # Compute loss.
        loss = loss_fn(x, reconstructed)
        loss += sum(vae.losses)  # Add KLD term.
    # Update the weights of the VAE.
    grads = tape.gradient(loss, vae.trainable_weights)
    optimizer.apply_gradients(zip(grads, vae.trainable_weights))
    return loss


losses = []  # Keep track of the losses over time.
for step, x in enumerate(dataset):
    loss = training_step(x)
    # Logging.
    losses.append(float(loss))
    if step % 100 == 0:
        print("Step:", step, "Loss:", sum(losses) / len(losses))

    # Stop after 1000 steps.
    # Training the model to convergence is left
    # as an exercise to the reader.
    if step >= 1000:
        break
        
original_dim = 784
intermediate_dim = 64
latent_dim = 32

# Define encoder model.
original_inputs = tf.keras.Input(shape=(original_dim,), name="encoder_input")
x = layers.Dense(intermediate_dim, activation="relu")(original_inputs)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()((z_mean, z_log_var))
encoder = tf.keras.Model(inputs=original_inputs, outputs=z, name="encoder")

# Define decoder model.
latent_inputs = tf.keras.Input(shape=(latent_dim,), name="z_sampling")
x = layers.Dense(intermediate_dim, activation="relu")(latent_inputs)
outputs = layers.Dense(original_dim, activation="sigmoid")(x)
decoder = tf.keras.Model(inputs=latent_inputs, outputs=outputs, name="decoder")

# Define VAE model.
outputs = decoder(z)
vae = tf.keras.Model(inputs=original_inputs, outputs=outputs, name="vae")

# Add KL divergence regularization loss.
kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
vae.add_loss(kl_loss)

# Loss and optimizer.
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Prepare a dataset.
(x_train, _), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    x_train.reshape(60000, 784).astype("float32") / 255
)
dataset = dataset.map(lambda x: (x, x))  # Use x_train as both inputs & targets
dataset = dataset.shuffle(buffer_size=1024).batch(32)

# Configure the model for training.
vae.compile(optimizer, loss=loss_fn)

# Actually training the model.
vae.fit(dataset, epochs=1)

# predict
predictions = vae.predict(dataset)
print(predictions.shape)

Step: 0 Loss: 0.36481335759162903
Step: 100 Loss: 0.12735670657441167
Step: 200 Loss: 0.10050674894852425
Step: 300 Loss: 0.09027601045825552
Step: 400 Loss: 0.08520745511104044
Step: 500 Loss: 0.0819876949765725
Step: 600 Loss: 0.07947824447851212
Step: 700 Loss: 0.07809196312123971
Step: 800 Loss: 0.07683713382549501
Step: 900 Loss: 0.07587328220363065
Step: 1000 Loss: 0.07491447665653267
Train for 1875 steps
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0714
(60000, 784)


In [6]:
# https://keras.io/getting_started/intro_to_keras_for_engineers/

# setup
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import layers

dataset = keras.preprocessing.text_dataset_from_directory(
  'path/to/main_directory', batch_size=64)

# For demonstration, iterate over the batches yielded by the dataset.
for data, labels in dataset:
    print(data.shape)  # (64,)
    print(data.dtype)  # string
    print(labels.shape)  # (64,)
    print(labels.dtype)  # int32

# Example training data, of dtype `string`.
training_data = np.array([["This is the 1st sample."], ["And here's the 2nd sample."]])

# Create a TextVectorization layer instance. It can be configured to either
# return integer token indices, or a dense token representation (e.g. multi-hot
# or TF-IDF). The text standardization and text splitting algorithms are fully
# configurable.
vectorizer = TextVectorization(output_mode="binary", ngrams=2)

# Calling `adapt` on an array or dataset makes the layer generate a vocabulary
# index for the data, which can then be reused when seeing new data.
vectorizer.adapt(training_data)

# After calling adapt, the layer is able to encode any n-gram it has seen before
# in the `adapt()` data. Unknown n-grams are encoded via an "out-of-vocabulary"
# token.
integer_data = vectorizer(training_data)
print(integer_data)

# Example image data, with values in the [0, 255] range
training_data = np.random.randint(0, 256, size=(64, 200, 200, 3)).astype("float32")

normalizer = Normalization(axis=-1)
normalizer.adapt(training_data)

normalized_data = normalizer(training_data)
print("var: %.4f" % np.var(normalized_data))
print("mean: %.4f" % np.mean(normalized_data))

# Example image data, with values in the [0, 255] range
training_data = np.random.randint(0, 256, size=(64, 200, 200, 3)).astype("float32")

cropper = CenterCrop(height=150, width=150)
scaler = Rescaling(scale=1.0 / 255)

output_data = scaler(cropper(training_data))
print("shape:", output_data.shape)
print("min:", np.min(output_data))
print("max:", np.max(output_data))

dense = keras.layers.Dense(units=16)

# Let's say we expect our inputs to be RGB images of arbitrary size
inputs = keras.Input(shape=(None, None, 3))

# Center-crop images to 150x150
x = CenterCrop(height=150, width=150)(inputs)
# Rescale images to [0, 1]
x = Rescaling(scale=1.0 / 255)(x)

# Apply some convolution and pooling layers
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)

# Apply global average pooling to get flat feature vectors
x = layers.GlobalAveragePooling2D()(x)

# Add a dense classifier on top
num_classes = 10
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

data = np.random.randint(0, 256, size=(64, 200, 200, 3)).astype("float32")
processed_data = model(data)
print(processed_data.shape)

model.summary()

model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.CategoricalCrossentropy())
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit(numpy_array_of_samples, numpy_array_of_labels,
          batch_size=32, epochs=10)

model.fit(dataset_of_samples_and_labels, epochs=10)

# Get the data as Numpy arrays
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Build a simple model
inputs = keras.Input(shape=(28, 28))
x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

# Train the model for 1 epoch from Numpy data
batch_size = 64
print("Fit on NumPy data")
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=1)

# Train the model for 1 epoch using a dataset
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
print("Fit on Dataset")
history = model.fit(dataset, epochs=1)

print(history.history)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)
history = model.fit(dataset, epochs=1)

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
history = model.fit(dataset, epochs=1, validation_data=val_dataset)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='path/to/my/model_{epoch}',
        save_freq='epoch')
]
model.fit(dataset, epochs=2, callbacks=callbacks)

callbacks = [
    keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(dataset, epochs=2, callbacks=callbacks)

# writes to: tensorboard --logdir=./logs

loss, acc = model.evaluate(val_dataset)  # returns loss and metrics
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)

predictions = model.predict(val_dataset)
print(predictions.shape)

# end-to-end example: https://keras.io/examples/nlp/text_classification_from_scratch/

ImportError: cannot import name 'CenterCrop'